# cifar-10 pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a `pipeline` to train a pytorch model based on cifar-10 dataset

**Motivations** - This notebook explains how to build pipeline to train a pytorch model. PyTorch is an open source machine learning framework, which is widely used for applications such as computer vision and natural language processing.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
#import required libraries
from azure.identity import InteractiveBrowserCredential
from azure.ml import MLClient, dsl
from azure.ml.entities import load_component, JobInput

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

# 2. Define command component via YAML
Define command component using YAML and load as function.

In [ ]:
parent_dir = '.'
get_data_func = load_component(yaml_file=parent_dir + "/get-data.yml")
train_model_func = load_component(yaml_file=parent_dir + "/train-model.yml")
eval_model_func = load_component(yaml_file=parent_dir + "/eval-model.yml")

# 3. Basic pipeline job

## 3.1 Build pipeline

In [ ]:
# Define pipeline
@dsl.pipeline(
    description="CIFAR-10 Pipeline Example",
    default_compute="cpu-cluster", 
    experiment_name="pipeline_samples"
)
def train_cifar_10_with_pytorch():
    #define the job to get data
    get_data = get_data_func(
        cifar_zip=JobInput(path="https://azuremlexamples.blob.core.windows.net/datasets/cifar-10-python.tar.gz")
    )

    #define the job to train the model
    train_model = train_model_func(epochs=1, cifar=get_data.outputs.cifar)
    train_model.compute = 'gpu-cluster'
    
    #define the job to evaluate the model
    eval_model = eval_model_func(cifar=get_data.outputs.cifar, model_dir = train_model.outputs.model_dir)
    eval_model.compute = 'gpu-cluster'

pipeline = train_cifar_10_with_pytorch()

# 3.2 Submit pipeline job

In [ ]:
#submit the pipeline job
pipeline_job = ml_client.create_or_update(pipeline)
#get a URL for the status of the job
print(f'Job link: {pipeline_job.services["Studio"].endpoint}')

In [ ]:
# Wait until the job completes
# ml_client.jobs.stream(returned_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)